## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/143-googleq-a-train-3fold-bert-base-unc-hidden10/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/143-googleq-a-train-3fold-bert-base-unc-hidden10/model_fold_1_last_epoch.h5
/kaggle/input/143-googleq-a-train-3fold-bert-base-unc-hidden10/model_fold_2_last_epoch.h5
/kaggle/input/143-googleq-a-train-3fold-bert-base-unc-hidden10/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
#     h12 = tf.reshape(hidden_states[-1][:,0],(-1,1,768))
#     h11 = tf.reshape(hidden_states[-2][:,0],(-1,1,768))
    h10 = tf.reshape(hidden_states[-3][:,0],(-1,1,768))
#     h09 = tf.reshape(hidden_states[-4][:,0],(-1,1,768))

    x = GlobalAveragePooling1D()(h10)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.960747,0.669075,0.225975,0.460784,0.558730,0.499155,0.626588,0.708372,0.604929,...,0.913120,0.929051,0.596829,0.975736,0.961140,0.834281,0.031195,0.028099,0.892692,0.916943
1,46,0.892115,0.567135,0.006598,0.781180,0.805907,0.931555,0.543885,0.471808,0.052293,...,0.698682,0.951068,0.626857,0.976405,0.980599,0.863360,0.916267,0.112347,0.074348,0.894832
2,70,0.904521,0.562098,0.019243,0.788756,0.904974,0.950675,0.587688,0.475028,0.107969,...,0.839736,0.943149,0.620090,0.976182,0.977517,0.862127,0.088125,0.046713,0.882602,0.899138
3,132,0.888186,0.460518,0.008971,0.743520,0.802447,0.913248,0.535693,0.479082,0.058085,...,0.708995,0.954503,0.676473,0.970440,0.985289,0.902647,0.800858,0.142659,0.704559,0.897521
4,200,0.918335,0.413682,0.067541,0.793176,0.636217,0.792307,0.658769,0.630439,0.081195,...,0.684721,0.885769,0.586428,0.959182,0.962452,0.807138,0.206477,0.087708,0.715311,0.894705


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.901836,0.597586,0.045255,0.745967,0.809059,0.835383,0.570856,0.499513,0.200739,...,0.798968,0.940075,0.657558,0.968103,0.972700,0.871386,0.513454,0.125651,0.528944,0.908126
std,2812.670060,0.044394,0.132830,0.083756,0.100596,0.114641,0.143268,0.052988,0.083385,0.183100,...,0.086976,0.023325,0.048445,0.011014,0.012116,0.041224,0.334432,0.068009,0.287809,0.020310
min,39.000000,0.754740,0.340475,0.003570,0.346006,0.232560,0.123943,0.457642,0.330793,0.005767,...,0.562449,0.833107,0.499237,0.924289,0.913045,0.700556,0.003408,0.006661,0.021770,0.827211
25%,2572.000000,0.872785,0.481915,0.009004,0.685422,0.775378,0.795121,0.532304,0.445093,0.067666,...,0.729689,0.930264,0.624016,0.962334,0.967250,0.848782,0.150286,0.070193,0.273675,0.895981
50%,5093.000000,0.902479,0.590916,0.014539,0.747869,0.822534,0.890958,0.561243,0.472810,0.125011,...,0.806226,0.945079,0.658811,0.970308,0.975092,0.879828,0.606276,0.126270,0.556776,0.909529
75%,7482.000000,0.940416,0.702438,0.034376,0.816504,0.879728,0.926698,0.602099,0.539758,0.277050,...,0.879329,0.955408,0.688552,0.975351,0.981010,0.901180,0.827653,0.175134,0.775379,0.921536
max,9640.000000,0.980702,0.896858,0.760388,0.969362,0.981362,0.975373,0.742717,0.764282,0.798085,...,0.951259,0.982919,0.789518,0.988482,0.991866,0.964232,0.965763,0.307209,0.988135,0.957948
